In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests

from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob
import spacy

In [2]:
def get_bbc():
    driver = webdriver.Chrome()
    url= 'https://www.bbc.com/news'
    driver.get(url)
    time.sleep(5)
    cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
    search_bar     = driver.find_element(By.CLASS_NAME, 'ux-v5').click()
    search_bar     = driver.find_element(By.ID, 'search-input')
    search_bar.send_keys('Ukraine')
    search_bar.send_keys(Keys.RETURN)
    lnks           = driver.find_elements(By.TAG_NAME,"a")
    lst_bbc        = [lnk.get_attribute('href') for lnk in lnks]
    lst_bbc_ukr    = [url for url in lst_bbc if '/news/' in url]
    return lst_bbc_ukr

In [3]:
def bbc_list(lst_bbc_ukr):
    dict_article_bbc={}
    for url in lst_bbc_ukr:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        article=soup.getText().replace('\'', "´").strip()
        dict_article_bbc[url]=article
    return dict_article_bbc

In [13]:
def descriptives(dict_article_bbc):
    length=[len(articles) for articles in dict_article_bbc.values()]
    lst_polarity = [round(TextBlob(article).sentiment.polarity,5) for article in dict_article_bbc.values()]
    lst_subjectivity = [round(TextBlob(article).sentiment.subjectivity,5) for article in dict_article_bbc.values()]
    ukraine=[articles.lower().count('ukraine') for articles in dict_article_bbc.values()]
    war=[articles.lower().count('war') for articles in dict_article_bbc.values()]
    russia=[articles.lower().count('russia') for articles in dict_article_bbc.values()]
    putin = [articles.lower().count('putin') for articles in dict_article_bbc.values()]
    zelensky= [articles.lower().count('zelensky') for articles in dict_article_bbc.values()]
    dict_length= {'polarity':lst_polarity,'subjetivity':lst_subjectivity,'length':length,'ukraine':ukraine,'war':war, 'russia':russia,'putin':putin,'zelensky':zelensky}
    df_descriptive= pd.DataFrame(dict_length)
    return df_descriptive

In [5]:
def create_df_bbc(dict_article_bbc):
    dict_article_bbc = bbc_list(lst_bbc_ukr)
    idx= {'article'}
    df_bbc= pd.DataFrame(dict_article_bbc,idx).transpose().reset_index()
    df_bbc= df_bbc.rename(columns={"index": "link"})
    return df_bbc

In [6]:
lst_bbc_ukr = get_bbc()
lst_bbc_ukr

['https://www.bbc.co.uk/news/world-asia-64773618',
 'https://www.bbc.co.uk/news/uk-england-wiltshire-64770144',
 'https://www.bbc.co.uk/news/uk-england-derbyshire-64736765',
 'https://www.bbc.co.uk/news/world-europe-64762219',
 'https://www.bbc.co.uk/news/blogs-the-papers-64766852',
 'https://www.bbc.co.uk/news/world-europe-64766558',
 'https://www.bbc.co.uk/news/uk-england-leeds-64760778',
 'https://www.bbc.co.uk/news/world-europe-jersey-64756351',
 'https://www.bbc.co.uk/news/uk-england-sussex-64757909',
 'https://www.bbc.co.uk/news/world-europe-isle-of-man-64758915',
 'https://www.bbc.co.uk/news/help-41670342']

In [9]:
dict_article_bbc=bbc_list(lst_bbc_ukr)
len(dict_article_bbc)

11

In [15]:
df_descriptive =descriptives(dict_article_bbc)


In [11]:
df_bbc = create_df_bbc(dict_article_bbc)

In [16]:
df_bbc_together = df_bbc.join(df_descriptive)

In [17]:
df_bbc_together

,link,article,polarity,subjetivity,length,ukraine,war,russia,putin,zelensky
0,https://www.bbc.co.uk/news/world-asia-64773618,China refuses to condemn Russia´s Ukraine inva...,0.01335,0.27805,6073,15,10,25,4,1
1,https://www.bbc.co.uk/news/uk-england-wiltshir...,Group to cycle the shape of Ukraine in fundrai...,-0.01519,0.33729,5778,16,6,6,1,0
2,https://www.bbc.co.uk/news/uk-england-derbyshi...,Ukraine war: ´Settling somewhere new is hard -...,0.05441,0.45468,4064,13,9,4,0,4
3,https://www.bbc.co.uk/news/world-europe-64762219,Ukraine war: Zelensky wants Xi Jinping meeting...,0.03261,0.36977,8274,24,18,22,8,10
4,https://www.bbc.co.uk/news/blogs-the-papers-64...,Newspaper headlines: ´PM to seal Brexit deal´ ...,0.01226,0.29627,3617,4,2,2,0,0
5,https://www.bbc.co.uk/news/world-europe-64766558,Ukraine war: Russia interrupts UN minute´s sil...,0.13333,0.25667,3500,15,9,6,2,4
6,https://www.bbc.co.uk/news/uk-england-leeds-64...,Ukraine invasion: Mother tells of arduous jour...,0.05235,0.43483,5502,16,9,4,1,0
7,https://www.bbc.co.uk/news/world-europe-jersey...,Jersey provides more donations to support Ukra...,-0.02918,0.37900,4205,9,5,4,1,0
8,https://www.bbc.co.uk/news/uk-england-sussex-6...,Ukraine war: Chef helps feed Ukrainian soldier...,0.08406,0.37742,5842,15,11,9,2,0
9,https://www.bbc.co.uk/news/world-europe-isle-o...,Russia-Ukraine war: Isle of Man marks invasion...,0.00032,0.33389,5602,16,7,7,1,0


# until here ok, forward testing

In [26]:
driver = webdriver.Chrome()
url= 'https://www.bbc.com/news'
driver.get(url)
time.sleep(2)
cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
search_bar     = driver.find_element(By.CLASS_NAME, 'ux-v5').click()
search_bar     = driver.find_element(By.ID, 'search-input')
search_bar.send_keys('Ukraine')
search_bar.send_keys(Keys.RETURN)



In [52]:
search_bar = driver.find_element(By.CLASS_NAME, 'ssrcss-u1ho06-StyledListItem-PageButtonListItem').click()
#page 2 ssrcss-hp3otd-StyledListItem-PageButtonListItem
#page 3 ssrcss-u1ho06-StyledListItem-PageButtonListItem

In [ ]:
element = driver.find_element(By.XPATH, "//input[@id='passwd-id']")